In [1]:
import os
import random
import sqlite3
import sys
import time
from datetime import date, datetime, timedelta
import pandas as pd
import re 

import toml
import requests
from tqdm import tqdm
from sbrscrape import Scoreboard
import numpy as np



In [2]:
data_headers = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Host': 'stats.nba.com',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
    'Referer': 'https://www.nba.com/',
    'Connection': 'keep-alive'
}


def to_data_frame(data):
    try:
        data_list = data[0]
    except Exception as e:
        print(e)
        return pd.DataFrame(data={})
    return pd.DataFrame(data=data_list.get('rowSet'), columns=data_list.get('headers'))

def get_json_data(url):
    raw_data = requests.get(url, headers=data_headers)
    try:
        json = raw_data.json()
    except Exception as e:
        print(e)
        return {}
    return json.get('resultSets')


def get_date(date_string):
    year1,month,day = re.search(r'(\d+)-\d+-(\d\d)(\d\d)', date_string).groups()
    year = year1 if int(month) > 8 else int(year1) + 1
    return datetime.strptime(f"{year}-{month}-{day}", '%Y-%m-%d')

def generate_seasons(from_year, to_year):
    seasons = []
    for year in range(int(from_year), int(to_year) + 1):
        season_str = f"{year}-{str((year + 1) % 100).zfill(2)}"
        seasons.append(season_str)
    return seasons

In [3]:
url = 'https://stats.nba.com/stats/' \
      'leaguedashteamstats?Conference=&' \
      'DateFrom=10%2F01%2F{2}&DateTo={0}%2F{1}%2F{3}' \
      '&Division=&GameScope=&GameSegment=&LastNGames=0&' \
      'LeagueID=00&Location=&MeasureType=Base&Month=0&' \
      'OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&' \
      'PerMode=PerGame&Period=0&PlayerExperience=&' \
      'PlayerPosition=&PlusMinus=N&Rank=N&' \
      'Season={4}' \
      '&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&' \
      'StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision='



In [40]:
sys.path.insert(1, os.path.join(sys.path[0], '..'))

config = toml.load("config.toml")
con = sqlite3.connect("Data/teams.sqlite")

## For each season in the list, go through each month and day and pull the game data

for key, value in config['get-data'].items():
    date_pointer = datetime.strptime(value['start_date'], "%Y-%m-%d").date()
    end_date = datetime.strptime(value['end_date'], "%Y-%m-%d").date()

    while date_pointer <= end_date:
        print("Getting data: ", date_pointer)

        raw_data = get_json_data(
            url.format(date_pointer.month, date_pointer.day, value['start_year'], date_pointer.year, key))
        df = to_data_frame(raw_data)

        date_pointer = date_pointer + timedelta(days=1)

        df['Date'] = str(date_pointer)

        df.to_sql(date_pointer.strftime("%Y-%m-%d"), con, if_exists="replace")

        time.sleep(random.randint(1, 3))

        # TODO: Add tests

con.close()

  0%|          | 0/15 [00:00<?, ?it/s]













100%|██████████| 31/31 [00:36<00:00,  1.18s/it]































100%|██████████| 31/31 [01:31<00:00,  2.95s/it]
































100%|██████████| 31/31 [01:24<00:00,  2.72s/it]
































100%|██████████| 31/31 [01:32<00:00,  2.98s/it]





























100%|██████████| 31/31 [01:20<00:00,  2.61s/it]
































100%|██████████| 31/31 [01:32<00:00,  2.99s/it]































100%|██████████| 31/31 [01:34<00:00,  3.04s/it]
































100%|██████████| 31/31 [01:42<00:00,  3.30s/it]































  7%|▋         | 1/15 [12:47<2:59:00, 767.17s/it]













100%|██████████| 31/31 [00:37<00:00,  1.20s/it]































100%|██████████| 31/31 [01:36<00:00,  3.11s/it]
































100%|██████████| 31/31 [01:40<00:00,  3.23s/it]
































100%|███████

Expecting value: line 1 column 1 (char 0)
0


Expecting value: line 1 column 1 (char 0)
0








 27%|██▋       | 4/15 [52:58<2:28:05, 807.79s/it]













100%|██████████| 31/31 [00:38<00:00,  1.25s/it]































100%|██████████| 31/31 [01:25<00:00,  2.77s/it]
































100%|██████████| 31/31 [01:25<00:00,  2.77s/it]
































100%|██████████| 31/31 [01:38<00:00,  3.17s/it]





























100%|██████████| 31/31 [01:26<00:00,  2.79s/it]
































100%|██████████| 31/31 [01:35<00:00,  3.08s/it]































100%|██████████| 31/31 [01:27<00:00,  2.83s/it]
































100%|██████████| 31/31 [01:36<00:00,  3.11s/it]































 33%|███▎      | 5/15 [1:05:46<2:12:16, 793.68s/it]












100%|██████████| 31/31 [00:42<00:00,  1.36s/it]































100%|██████████| 31/31 [01:31<00:00,  2.95s/it]
































100%|██████████| 31/31 [01:35<00:00,  3.09s/it]




























Expecting value: line 1 column 1 (char 0)
0


...

'NoneType' object is not subscriptable


'NoneType' object is not subscriptable


'NoneType' object is not subscriptable


'NoneType' object is not subscriptable



100%|██████████| 15/15 [3:14:06<00:00, 776.44s/it]


## Section 2. Pulling Odds Data and Joining to Odds Master

In [5]:
sportsbook = 'fanduel'
df_data = []

config = toml.load("config.toml")

con = sqlite3.connect("Data/oddsData.sqlite")

for key, value in config['get-odds-data'].items():
    date_pointer = datetime.strptime(value['start_date'], "%Y-%m-%d").date()
    end_date = datetime.strptime(value['end_date'], "%Y-%m-%d").date()
    teams_last_played = {}

    while date_pointer <= end_date:
        print("Getting odds data: ", date_pointer)
        sb = Scoreboard(date=date_pointer)

        if not hasattr(sb, "games"):
            date_pointer = date_pointer + timedelta(days=1)
            continue

        for game in sb.games:
            if game['home_team'] not in teams_last_played:
                teams_last_played[game['home_team']] = date_pointer
                home_games_rested = timedelta(days=7)  # start of season, big number
            else:
                current_date = date_pointer
                home_games_rested = current_date - teams_last_played[game['home_team']]
                teams_last_played[game['home_team']] = current_date

            if game['away_team'] not in teams_last_played:
                teams_last_played[game['away_team']] = date_pointer
                away_games_rested = timedelta(days=7)  # start of season, big number
            else:
                current_date = date_pointer
                away_games_rested = current_date - teams_last_played[game['away_team']]
                teams_last_played[game['away_team']] = current_date

            try:
                df_data.append({
                    'Date': date_pointer,
                    'Home': game['home_team'],
                    'Away': game['away_team'],
                    'OU': game['total'][sportsbook],
                    'Spread': game['away_spread'][sportsbook],
                    'ML_Home': game['home_ml'][sportsbook],
                    'ML_Away': game['away_ml'][sportsbook],
                    'Points': game['away_score'] + game['home_score'],
                    'Win_Margin': game['home_score'] - game['away_score'],
                    'Days_Rest_Home': home_games_rested.days,
                    'Days_Rest_Away': away_games_rested.days
                })
            except KeyError:
                print(f"No {sportsbook} odds data found for game: {game}")

        date_pointer = date_pointer + timedelta(days=1)
        time.sleep(random.randint(1, 3))

    df = pd.DataFrame(df_data, )
    df.to_sql(key, con, if_exists="replace")
con.close()

Getting odds data:  2014-10-27
Getting odds data:  2014-10-28
No fanduel odds data found for game: {'date': '2014-10-29T00:00:00+00:00', 'status': 'Final', 'home_team': 'San Antonio Spurs', 'home_team_loc': 'San Antonio', 'home_team_abbr': 'SA', 'home_team_rank': -1, 'away_team': 'Dallas Mavericks', 'away_team_loc': 'Dallas', 'away_team_abbr': 'DAL', 'away_team_rank': -1, 'home_score': 101, 'away_score': 100, 'home_spread': {}, 'home_spread_odds': {}, 'away_spread': {}, 'away_spread_odds': {}, 'under_odds': {}, 'over_odds': {}, 'total': {}, 'home_ml': {}, 'away_ml': {}}
No fanduel odds data found for game: {'date': '2014-10-29T00:00:00+00:00', 'status': 'Final', 'home_team': 'New Orleans Pelicans', 'home_team_loc': 'New Orleans', 'home_team_abbr': 'NO', 'home_team_rank': -1, 'away_team': 'Orlando Magic', 'away_team_loc': 'Orlando', 'away_team_abbr': 'ORL', 'away_team_rank': -1, 'home_score': 101, 'away_score': 84, 'home_spread': {}, 'home_spread_odds': {}, 'away_spread': {}, 'away_spre

KeyboardInterrupt: 

## Section 3. Joining Odds and Game Log Data to Master

In [13]:
from Util.Dictionaries import team_index_07, team_index_08, team_index_12, team_index_13, team_index_14, \
    team_index_current

df = pd.DataFrame
scores = []
win_margin = []
OU = []
OU_Cover = []
games = []
days_rest_away = []
days_rest_home = []
teams_con = sqlite3.connect("Data/TeamData.sqlite")
odds_con = sqlite3.connect("Data/OddsData.sqlite")

for key, value in config['create-games'].items():
    print(key)
    odds_df = pd.read_sql_query(f"select * from \"odds_{key}_new\"", odds_con, index_col="index")
    team_table_str = key
    year_count = 0
    season = key

    for row in odds_df.itertuples():
        home_team = row[2]
        away_team = row[3]

        date = row[1]

        team_df = pd.read_sql_query(f"select * from \"{date}\"", teams_con, index_col="index")
        if len(team_df.index) == 30:
            scores.append(row[8])
            OU.append(row[4])
            days_rest_home.append(row[10])
            days_rest_away.append(row[11])
            if row[9] > 0:
                win_margin.append(1)
            else:
                win_margin.append(0)

            if row[8] < row[4]:
                OU_Cover.append(0)
            elif row[8] > row[4]:
                OU_Cover.append(1)
            elif row[8] == row[4]:
                OU_Cover.append(2)

            if season == '2007-08':
                home_team_series = team_df.iloc[team_index_07.get(home_team)]
                away_team_series = team_df.iloc[team_index_07.get(away_team)]
            elif season == '2008-09' or season == "2009-10" or season == "2010-11" or season == "2011-12":
                home_team_series = team_df.iloc[team_index_08.get(home_team)]
                away_team_series = team_df.iloc[team_index_08.get(away_team)]
            elif season == "2012-13":
                home_team_series = team_df.iloc[team_index_12.get(home_team)]
                away_team_series = team_df.iloc[team_index_12.get(away_team)]
            elif season == '2013-14':
                home_team_series = team_df.iloc[team_index_13.get(home_team)]
                away_team_series = team_df.iloc[team_index_13.get(away_team)]
            elif season == '2022-23' or season == '2023-24':
                home_team_series = team_df.iloc[team_index_current.get(home_team)]
                away_team_series = team_df.iloc[team_index_current.get(away_team)]
            else:
                try:
                    home_team_series = team_df.iloc[team_index_14.get(home_team)]
                    away_team_series = team_df.iloc[team_index_14.get(away_team)]
                except Exception as e:
                    print(home_team)
                    raise e
            game = pd.concat([home_team_series, away_team_series.rename(
                index={col: f"{col}.1" for col in team_df.columns.values}
            )])
            games.append(game)
odds_con.close()
teams_con.close()
season = pd.concat(games, ignore_index=True, axis=1)
season = season.T
frame = season.drop(columns=['TEAM_ID', 'TEAM_ID.1'])
frame['Score'] = np.asarray(scores)
frame['Home-Team-Win'] = np.asarray(win_margin)
frame['OU'] = np.asarray(OU)
frame['OU-Cover'] = np.asarray(OU_Cover)
frame['Days-Rest-Home'] = np.asarray(days_rest_home)
frame['Days-Rest-Away'] = np.asarray(days_rest_away)
# fix types
for field in frame.columns.values:
    if 'TEAM_' in field or 'Date' in field or field not in frame:
        continue
    frame[field] = frame[field].astype(float)
con = sqlite3.connect("Data/dataset.sqlite")
frame.to_sql("dataset_2012-24_new", con, if_exists="replace")
con.close()

2012-13
2013-14
2014-15
2015-16
2016-17
2017-18
2018-19
2019-20
2020-21
2021-22
2022-23
2023-24


## Section 4. Building Player Game Log DB

In [4]:
## Create Dataframe of teams in NBA + ID values for querying.
# TODO Save this as CSV or add to SQLite DB instead of querying new df each time

from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.static import teams
from nba_api.stats.endpoints import boxscoretraditionalv3

import pandas as pd


nba_teams = teams.get_teams()
team_df = pd.DataFrame(nba_teams)

print(team_df.head())

           id             full_name abbreviation   nickname         city  \
0  1610612737         Atlanta Hawks          ATL      Hawks      Atlanta   
1  1610612738        Boston Celtics          BOS    Celtics       Boston   
2  1610612739   Cleveland Cavaliers          CLE  Cavaliers    Cleveland   
3  1610612740  New Orleans Pelicans          NOP   Pelicans  New Orleans   
4  1610612741         Chicago Bulls          CHI      Bulls      Chicago   

           state  year_founded  
0        Georgia          1949  
1  Massachusetts          1946  
2           Ohio          1970  
3      Louisiana          2002  
4       Illinois          1966  


In [5]:
from nba_api.stats.endpoints import playergamelogs

## Example code for querying player game log data. 
test= playergamelogs.PlayerGameLogs(player_id_nullable = '203076', season_nullable= ['2022-23','2023-24'], season_type_nullable='Regular Season')

test = test.get_data_frames()[0]
print(test)



   SEASON_YEAR  PLAYER_ID    PLAYER_NAME NICKNAME     TEAM_ID  \
0      2022-23     203076  Anthony Davis  Anthony  1610612747   
1      2022-23     203076  Anthony Davis  Anthony  1610612747   
2      2022-23     203076  Anthony Davis  Anthony  1610612747   
3      2022-23     203076  Anthony Davis  Anthony  1610612747   
4      2022-23     203076  Anthony Davis  Anthony  1610612747   
5      2022-23     203076  Anthony Davis  Anthony  1610612747   
6      2022-23     203076  Anthony Davis  Anthony  1610612747   
7      2022-23     203076  Anthony Davis  Anthony  1610612747   
8      2022-23     203076  Anthony Davis  Anthony  1610612747   
9      2022-23     203076  Anthony Davis  Anthony  1610612747   
10     2022-23     203076  Anthony Davis  Anthony  1610612747   
11     2022-23     203076  Anthony Davis  Anthony  1610612747   
12     2022-23     203076  Anthony Davis  Anthony  1610612747   
13     2022-23     203076  Anthony Davis  Anthony  1610612747   
14     2022-23     203076

In [6]:
from nba_api.stats.endpoints import boxscoreadvancedv3

id = '0022301195'
test_adv= boxscoreadvancedv3.BoxScoreAdvancedV3(game_id=id) 

print(test_adv.get_data_frames()[0])
#boxscoreadvancedv3.BoxScoreAdvancedV3(game_id=id) 


        gameId      teamId     teamCity  teamName teamTricode  teamSlug  \
0   0022301195  1610612747  Los Angeles    Lakers         LAL    lakers   
1   0022301195  1610612747  Los Angeles    Lakers         LAL    lakers   
2   0022301195  1610612747  Los Angeles    Lakers         LAL    lakers   
3   0022301195  1610612747  Los Angeles    Lakers         LAL    lakers   
4   0022301195  1610612747  Los Angeles    Lakers         LAL    lakers   
5   0022301195  1610612747  Los Angeles    Lakers         LAL    lakers   
6   0022301195  1610612747  Los Angeles    Lakers         LAL    lakers   
7   0022301195  1610612747  Los Angeles    Lakers         LAL    lakers   
8   0022301195  1610612747  Los Angeles    Lakers         LAL    lakers   
9   0022301195  1610612747  Los Angeles    Lakers         LAL    lakers   
10  0022301195  1610612747  Los Angeles    Lakers         LAL    lakers   
11  0022301195  1610612747  Los Angeles    Lakers         LAL    lakers   
12  0022301195  161061274

Framework for building NBA game log DB 

Task #1 
Create dictionary of player IDs (designate active v inactive) DONE

Task #2 
Use dictionary to go through by year and by player and add game logs to SQLite DB DONE
    Start with regular season and then add playoffs (use playergamelogs function)
    Expand with advanced stats 

Expand with Career/season stats page

In [7]:
from nba_api.stats.static import players
# get_players returns a list of dictionaries, each representing a player.
nba_players = players.get_players()
print('Number of players fetched: {}'.format(len(nba_players)))
player_id_df = pd.DataFrame(nba_players)
player_id_df['GamesQueried'] = 0

print(player_id_df['id'])


from nba_api.stats.endpoints import commonplayerinfo


common_player_df = pd.DataFrame()

#for id in player_id_df['id']:
#    t2 = commonplayerinfo.CommonPlayerInfo(player_id=id)
#    t2 = t2.get_data_frames()
#    common_player_df = common_player_df.append(t2)
#    time.sleep(random.randint(2, 5))
    

print(common_player_df.head())

Number of players fetched: 4940
0         76001
1         76002
2         76003
3            51
4          1505
         ...   
4935    1627790
4936      78647
4937      78648
4938    1627826
4939      78650
Name: id, Length: 4940, dtype: int64
Empty DataFrame
Columns: []
Index: []


In [8]:
#player_id_df.to_csv('player_list.csv')

player_id_df = pd.read_csv('player_list.csv')

In [14]:
from nba_api.stats.endpoints import commonallplayers

t_common_2 = commonallplayers.CommonAllPlayers(season='2023-24')
### Also includes inactive players so if this is used need to 

df_2 =t_common_2.get_data_frames()[0]


df_2.to_csv("CommonAllPlayerTest.csv")

player_common = player_id_df.merge(df_2, left_on= 'id', right_on='PERSON_ID' ,how='left')

con = sqlite3.connect("Data/player_data.sqlite")

player_common.to_sql("player_list", con, if_exists="replace")
con.close()

In [21]:
from nba_api.stats.endpoints import playergamelogs

### Need to relabel summer league games based on game ID
### Need to create a basic framework for adding on advanced stats to this base table. 
### Should add master game log table to database. 

con = sqlite3.connect("Data/player_data.sqlite")

cur = con.cursor()

cur.execute("SELECT id, FROM_YEAR, TO_YEAR, GamesQueried FROM player_list")

player_id_years = cur.fetchall()



for row in player_id_years:

    if row[3] == 1:
        print("Already Collected" + str(row))
        continue

    if row[1] == None:
        print("No Year Available"+ str(row))
        continue
    print(row) 

    id = row[0]
    seasons = generate_seasons(row[1], row[2])
    player_data = pd.DataFrame()

    ### Update DB to show that games for this player have been queried
    cur.execute("UPDATE player_list SET GamesQueried = 1 WHERE id = ?", (id,))

    for season in seasons:
        test= playergamelogs.PlayerGameLogs(player_id_nullable = id, season_nullable= season, season_type_nullable='Regular Season')
        player_season = test.get_data_frames()[0]
        player_season['Season_Type'] = 'Regular Season'

        player_data = pd.concat([player_data, player_season], ignore_index=True)
        time.sleep(0.6)

        #print(player_season.head())
    
    player_data.to_sql(f"player_{id}", con, if_exists="replace")
    



con.commit()


con.close()


Already Collected(76001, '1990', '1994', 1)
Already Collected(76002, '1968', '1977', 1)
Already Collected(76003, '1969', '1988', 1)
Already Collected(51, '1990', '2000', 1)
Already Collected(1505, '1997', '2003', 1)
Already Collected(949, '1996', '2007', 1)
Already Collected(76005, '1976', '1980', 1)
Already Collected(76006, '1956', '1956', 1)
Already Collected(76007, '1946', '1947', 1)
Already Collected(203518, '2016', '2018', 1)
Already Collected(1630173, '2020', '2024', 1)
Already Collected(101165, '2005', '2008', 1)
Already Collected(76008, '1953', '1953', 1)
Already Collected(76009, '1987', '1992', 1)
Already Collected(76010, '1967', '1967', 1)
Already Collected(203112, '2012', '2018', 1)
Already Collected(76011, '1975', '1987', 1)
Already Collected(76012, '1970', '1976', 1)
Already Collected(200801, '2006', '2008', 1)
Already Collected(1629121, '2018', '2020', 1)
Already Collected(203919, '2014', '2015', 1)
Already Collected(149, '1985', '1995', 1)
Already Collected(203500, '2013

In [66]:
## TODO Join Advanced box score stats to game logs 

## Get List of Game IDs (should be reproducable for when season starts back up in october)
## Create table in DB for game IDs



games_master = pd.DataFrame()


## USING THIS AS MASTER LIST OF GAMES PLAYED NEED TO EXCLUDE SL GAMES
## Pulls summer league games as well as playoffs
for team_id in team_df['id']:
    games = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id, season_type_nullable= 'Regular Season' )
    games_df = games.get_data_frames()[0]
    games_master = pd.concat([games_master,games_df], ignore_index=True)
    time.sleep(0.6)

## Adding label to differentiate Summer League, Regular Season and Playoffs
games_master['GameType'] = 'Regular Season'
games_master.loc[games_master['GAME_ID'].str.startswith(('15', '16', '13')), 'GameType'] = 'Summer League'

playoffs = pd.DataFrame()

for team_id in team_df['id']:
    games = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id, season_type_nullable= 'Playoffs' )
    games_df = games.get_data_frames()[0]
    playoffs = pd.concat([playoffs,games_df], ignore_index=True)
    time.sleep(0.6)

playoffs['GameType'] = 'Playoffs'

## Joining Playoffs and Regular Season/Summer League Dataframes 
games_master = pd.concat([games_master,playoffs], ignore_index=True)


## Adding this table to Database 
con = sqlite3.connect("Data/player_data.sqlite")
games_master.to_sql("MasterGameList", con, if_exists="replace")
con.close()


In [57]:
## TODO Join Advanced box score stats to game logs 
id = 1629635
#id = 203076
season = '2019-20'

test= playergamelogs.PlayerGameLogs(player_id_nullable = id, season_nullable= season, season_type_nullable='Regular Season')
player_season = test.get_data_frames()[0]

print(player_season.head())




  SEASON_YEAR  PLAYER_ID      PLAYER_NAME NICKNAME     TEAM_ID  \
0     2019-20    1629635  Sekou Doumbouya    Sekou  1610612765   
1     2019-20    1629635  Sekou Doumbouya    Sekou  1610612765   
2     2019-20    1629635  Sekou Doumbouya    Sekou  1610612765   
3     2019-20    1629635  Sekou Doumbouya    Sekou  1610612765   
4     2019-20    1629635  Sekou Doumbouya    Sekou  1610612765   

  TEAM_ABBREVIATION        TEAM_NAME     GAME_ID            GAME_DATE  \
0               DET  Detroit Pistons  0021900971  2020-03-11T00:00:00   
1               DET  Detroit Pistons  0021900955  2020-03-08T00:00:00   
2               DET  Detroit Pistons  0021900941  2020-03-07T00:00:00   
3               DET  Detroit Pistons  0021900918  2020-03-04T00:00:00   
4               DET  Detroit Pistons  0021900899  2020-03-01T00:00:00   

       MATCHUP  ... PF_RANK  PFD_RANK  PTS_RANK  PLUS_MINUS_RANK  \
0    DET @ PHI  ...      17        22        30               34   
1    DET @ NYK  ...       8 

In [40]:
row =(51, '1990', '2000')
seasons = generate_seasons(row[1], row[2])
print(seasons)
for season in seasons:
    test= playergamelogs.PlayerGameLogs(player_id_nullable = 51, season_nullable= season, season_type_nullable='Regular Season')

    test.get_data_frames()[0]


['1990-91', '1991-92', '1992-93', '1993-94', '1994-95', '1995-96', '1996-97', '1997-98', '1998-99', '1999-100', '2000-1']


KeyError: 'resultSet'

In [55]:
## Test code for game log queries

test= playergamelogs.PlayerGameLogs(player_id_nullable = '76002', season_nullable= '1968-69', season_type_nullable='Regular Season')

test = test.get_data_frames()[0]
print(test)

   SEASON_ID  PLAYER_ID      PLAYER_NAME NICKNAME     TEAM_ID  \
0      21968      76002  Zaid Abdul-Aziz     Zaid  1610612749   
1      21968      76002  Zaid Abdul-Aziz     Zaid  1610612749   
2      21968      76002  Zaid Abdul-Aziz     Zaid  1610612749   
3      21968      76002  Zaid Abdul-Aziz     Zaid  1610612749   
4      21968      76002  Zaid Abdul-Aziz     Zaid  1610612749   
5      21968      76002  Zaid Abdul-Aziz     Zaid  1610612749   
6      21968      76002  Zaid Abdul-Aziz     Zaid  1610612749   
7      21968      76002  Zaid Abdul-Aziz     Zaid  1610612749   
8      21968      76002  Zaid Abdul-Aziz     Zaid  1610612749   
9      21968      76002  Zaid Abdul-Aziz     Zaid  1610612749   
10     21968      76002  Zaid Abdul-Aziz     Zaid  1610612749   
11     21968      76002  Zaid Abdul-Aziz     Zaid  1610612749   
12     21968      76002  Zaid Abdul-Aziz     Zaid  1610612749   
13     21968      76002  Zaid Abdul-Aziz     Zaid  1610612749   
14     21968      76002  